# In situ validation data

Download data corresponding to LAI measurements

In [55]:
import os
from pathlib import Path
import sys
base_dir = Path(os.path.dirname(os.path.realpath("__file__"))).parent.parent
sys.path.insert(0, os.path.join(base_dir, "eodal"))
import eodal
from eodal.config import get_settings
from eodal.core.scene import SceneCollection
from eodal.core.sensors.sentinel2 import Sentinel2
from eodal.mapper.feature import Feature
from eodal.mapper.filter import Filter
from eodal.mapper.mapper import Mapper, MapperConfigs

Settings = get_settings()
# set to False to use a local data archive
Settings.USE_STAC = True

import geopandas as gpd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import contextily as ctx
import pandas as pd
from scipy.spatial import cKDTree
from shapely.geometry import Point
from geopy.distance import geodesic
from pyproj import Proj, transform

In [2]:
lai_path = base_dir.joinpath('data/in-situ_glai.gpkg')
gdf = gpd.read_file(lai_path)
gdf.head()

,date,lai,location,protocol_version,Barcode Biomass Sample,Barcode Leaf Area Sample,Row Spacing [cm],leaf_area_cm2,Comment,tmean_degC,gdd,gdd_cumsum,parcel,genotype,geometry
0,2022-03-17 10:34:56,0.447264,Arenenberg,v2.0,13508.0,13541,12.0,107.343424,None,8.6,8.6,694.5,Broatefaeld,None,POINT (1009573.514 6051626.822)
1,2022-03-28 09:07:15,0.331586,Arenenberg,v2.0,13574.0,13637,13.0,43.106220,2nd fertilizer application (N),11.6,11.6,802.1,Broatefaeld,None,POINT (1009584.963 6051647.736)
2,2022-05-11 09:48:44,2.770000,Arenenberg,v3.0,14970.0,0,14.0,0.000000,None,20.9,20.9,1287.6,Broatefaeld,None,POINT (1009582.481 6051635.184)
3,2022-05-11 09:48:44,2.770000,Arenenberg,v3.0,14970.0,0,14.0,0.000000,None,20.9,20.9,1287.6,Broatefaeld,None,POINT (1009582.481 6051635.184)
4,2022-05-19 13:14:04,3.670000,Arenenberg,v3.0,15096.0,0,13.0,0.000000,None,21.6,21.6,1447.6,Broatefaeld,None,POINT (1009579.299 6051649.448)


In [3]:
# Keep only locations of interest
gdf = gdf[gdf.location.isin(['Strickhof', 'SwissFutureFarm', 'Witzwil'])]

In [27]:
# Get S2 spectra for pixels if they are cloud free
# Craete pairs lai-spectra
# Apply models and compute RMSE

In [4]:
def preprocess_sentinel2_scenes(
    ds: Sentinel2,
    target_resolution: int,
) -> Sentinel2:
    """
    Resample Sentinel-2 scenes and mask clouds, shadows, and snow
    based on the Scene Classification Layer (SCL).

    NOTE:
        Depending on your needs, the pre-processing function can be
        fully customized using the full power of EOdal and its
    interfacing libraries!

    :param target_resolution:
        spatial target resolution to resample all bands to.
    :returns:
        resampled, cloud-masked Sentinel-2 scene.
    """
    # resample scene
    ds.resample(inplace=True, target_resolution=target_resolution)
    # mask clouds, shadows, and snow
    ds.mask_clouds_and_shadows(inplace=True, cloud_classes=[3, 8, 9, 10, 11])
    return ds
    

def extract_s2_data(
        aoi: gpd.GeoDataFrame,
        time_start: datetime,
        time_end: datetime,
        scene_cloud_cover_threshold: float = 80,
        feature_cloud_cover_threshold: float = 50,
        spatial_resolution: int = 10
    ) -> SceneCollection:
    """
    Extracts Sentinel-2 data from the STAC SAT archive for a given area and time period.
    Scenes that are too cloudy or contain nodata (blackfill), only, are discarded.

    The processing level of the Sentinel-2 data is L2A (surface reflectance factors).

    :param parcel:
        field parcel geometry (defines the spatial extent to extract)
    :param time_start:
        start of the time period to extract
    :param end_time:
        end of the time period to extract
    :param scene_cloud_cover_threshold:
        scene-wide cloudy pixel percentage (from Sentinel-2 metadata) to filter out scenes
        with too high cloud coverage values [0-100%]
    :param feature_cloud_cover_threshold:
        cloudy pixel percentage [0-100%] on the parcel level. Only if the parcel has a
        lower percentual share of cloudy pixels (based on the scene classification layer) than
        the threshold specified, the Sentinel-2 observation is kept
    :param spatial_resolution:
        spatial resolution of the Sentinel-2 data in meters (Def: 10m)
    :param resampling_method:
        spatial resampling method for those Sentinel-2 bands not available in the target
        resolution. Nearest Neighbor by default
    :returns:
        dictionary with the list of scenes for the field parcel (`feature_scenes`), the
        DataFrame of (un)used scenes and the reason for not using plus some basic scene
        metadata (`scene_properties`)
    """
    # setup the metadata filters (cloud cover and processing level)
    metadata_filters = [
        Filter('cloudy_pixel_percentage','<', scene_cloud_cover_threshold),
        Filter('processing_level', '==', 'Level-2A')
    ]
    # setup the spatial feature for extracting data
    feature = Feature.from_geoseries(aoi.geometry)
    
    # set up mapping configs
    mapper_configs = MapperConfigs(
        collection='sentinel2-msi',
        time_start=time_start,
        time_end=time_end,
        feature=feature,
        metadata_filters=metadata_filters
    )

    # get a new mapper instance. Set sensor to `sentinel2`
    mapper = Mapper(mapper_configs)

    # query the STAC (looks for available scenes in the selected spatio-temporal range)
    mapper.query_scenes()

    # get observations (loads the actual Sentinel2 scenes)
    # the data is extract for the extent of the parcel
    scene_kwargs = {
        'scene_constructor': Sentinel2.from_safe,            # this tells the mapper how to read and load the data (i.e., Sentinel-2 scenes)
        'scene_constructor_kwargs': {},                      # here you could specify which bands to read
        'scene_modifier': preprocess_sentinel2_scenes,       # this tells the mapper about (optional) pre-processing of the loaded scenes (must be a callable)
        'scene_modifier_kwargs': {'target_resolution': 10}   # here, you have to specify the value of the arguments the `scene_modifier` requires
    }
    mapper.load_scenes(scene_kwargs=scene_kwargs)

    # loop over available Sentinel-2 scenes stored in mapper.data as a EOdal SceneCollection and check
    # for no-data. These scenes will be removed from the SceneCollection
    scenes_to_del = []
    mapper.metadata['scene_used'] = 'yes'

    if mapper.data is not None:
        for scene_id, scene in mapper.data:

            # check if scene is blackfilled (nodata); if yes continue
            if scene.is_blackfilled:
                scenes_to_del.append(scene_id)
                mapper.metadata.loc[mapper.metadata.sensing_time.dt.strftime('%Y-%m-%d %H:%M') == scene_id.strftime('%Y-%m-%d %H:%M')[0:16], 'scene_used'] = 'No [blackfill]'
                continue

            # check cloud coverage (including shadows and snow) of the field parcel
            feature_cloud_cover = scene.get_cloudy_pixel_percentage(cloud_classes=[3, 8, 9, 10, 11])

            # if the scene is too cloudy, we skip it
            if feature_cloud_cover > feature_cloud_cover_threshold:
                scenes_to_del.append(scene_id)
                mapper.metadata.loc[mapper.metadata.sensing_time.dt.strftime('%Y-%m-%d %H:%M') == scene_id.strftime('%Y-%m-%d %H:%M')[0:16], 'scene_used'] = 'No [clouds]'
                continue

        # delete scenes containing only no-data
        for scene_id in scenes_to_del:
            del mapper.data[scene_id]

    
    return mapper

In [6]:
s2_data = extract_s2_data(
  aoi=gdf.head(2).dissolve(),
  time_start=datetime(2017,3,1),
  time_end=datetime(2017,5,31)
)

2024-02-02 17:34:16,804 eodal        INFO     Starting extraction of sentinel2 scenes


None


2024-02-02 17:34:41,911 eodal        INFO     Finished extraction of sentinel2 scenes


In [7]:
for scene_id, scene in s2_data.data:
  pixs = scene.to_dataframe()
  break

In [68]:
gdf_date = gdf.head(2)
gdf_date = gdf_date.to_crs('EPSG:4326')
pixs = pixs.to_crs('EPSG:4326')

In [69]:
def find_nearest(point, tree):
    ''' 
    Find the nearest point in pixs for each point in gdf_date
    '''
    _, index = tree.query((point.x, point.y))
    return pixs.iloc[index]

def compute_distance(row):
    return geodesic((row['geometry'].y, row['geometry'].x), (row['geometry_s2'].y, row['geometry_s2'].x)).meters

In [99]:
# Loop over dates of val data
# Query s2 data for those dates
# Keep pixel closest to geom

val_df = pd.DataFrame()

for d in gdf['date']:
  try:
    s2_data = extract_s2_data(
      aoi=gdf.dissolve(),
      time_start=pd.to_datetime(d), 
      time_end=pd.to_datetime(d) + timedelta(days=1)
    )
  except:
    pass
    
  if s2_data.data is not None:
    for scene_id, scene in s2_data.data:
      pixs = scene.to_dataframe()
      pixs = pixs.to_crs('EPSG:4326')
      
      # Find S2-data closest (within 10m) to validation data for that date
      gdf_date = gdf[gdf['date'] == d]
      gdf_date = gdf.date.to_crs(pixs.crs) # to meters, EPSG:32632

      pixs_tree = cKDTree(pixs['geometry'].apply(lambda geom: (geom.x, geom.y)).tolist())

      # Apply the function to each point in gdf_date
      nearest_points = gdf_date['geometry'].apply(lambda point: find_nearest(point, pixs_tree))
      nearest_points = nearest_points.rename(columns={'geometry': 'geometry_s2'})
      gdf_date = pd.concat([gdf_date[['date', 'lai', 'location', 'geometry']], nearest_points], axis=1)

      # # Keep if less than 10m away to ensure its the right pixel
      gdf_date['distance'] = gdf_date.apply(compute_distance, axis=1)      
      gdf_date = gdf_date[gdf_date['distance'] <10]
      
      # Save lai and spectra 
      if (len(gdf_date)):
        val_df = pd.concat([val_df, gdf_date[['lai', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12',]]])



# Save in-situ val data
data_path = base_dir.joinpath(f'results/validation_data.pkl')
with open(data_path, 'wb+') as dst:
    pickle.dump(val_df, dst)


/mnt/eo-nas1/eoa-share/projects/010_CropCovEO/eodal/eodal/mapper/mapper.py:887: UserWarning: No scenes were found - consider modifying your search criteria
  warnings.warn(
/mnt/eo-nas1/eoa-share/projects/010_CropCovEO/eodal/eodal/mapper/mapper.py:887: UserWarning: No scenes were found - consider modifying your search criteria
  warnings.warn(
/mnt/eo-nas1/eoa-share/projects/010_CropCovEO/eodal/eodal/mapper/mapper.py:887: UserWarning: No scenes were found - consider modifying your search criteria
  warnings.warn(
2024-02-02 17:19:02,610 eodal        INFO     Starting extraction of sentinel2 scenes


None


2024-02-02 17:30:54,202 eodal        INFO     Starting extraction of sentinel2 scenes


None
